In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler

In [3]:
# Load datasets
userbase = pd.read_csv('/Users/jamesmoy/Desktop/Analytics Portfolio/Netflix Churn Model/Netflix Userbase.csv', encoding='latin1', on_bad_lines='skip')
clickstream = pd.read_csv('/Users/jamesmoy/Desktop/Analytics Portfolio/Netflix Churn Model/WatchLog Data/All_Clickstream.csv', encoding='latin1', on_bad_lines='skip')
devices = pd.read_csv('/Users/jamesmoy/Desktop/Analytics Portfolio/Netflix Churn Model/WatchLog Data/All_Devices.csv', encoding='latin1', on_bad_lines='skip')
search_history = pd.read_csv('/Users/jamesmoy/Desktop/Analytics Portfolio/Netflix Churn Model/WatchLog Data/All_SearchHistory.csv', encoding='latin1', on_bad_lines='skip')
viewing_activity = pd.read_csv('/Users/jamesmoy/Desktop/Analytics Portfolio/Netflix Churn Model/WatchLog Data/All_ViewingActivity.csv', encoding='latin1', on_bad_lines='skip')

In [4]:
# Standardize column names
for dataset in [userbase, clickstream, devices, search_history, viewing_activity]:
    dataset.columns = dataset.columns.str.strip().str.lower()

In [5]:
# Netflix Userbase Analysis
print("\nNetflix Userbase Analysis:")
userbase['last_payment_date'] = pd.to_datetime(userbase['last payment date'], errors='coerce')
userbase['days_since_last_payment'] = (pd.Timestamp('2025-01-01') - userbase['last_payment_date']).dt.days
userbase['churn'] = (userbase['days_since_last_payment'] > 30).astype(int)



Netflix Userbase Analysis:


/var/folders/w6/vkcmr8l91_12vz93c0c46mtm0000gn/T/ipykernel_46806/1564370386.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  userbase['last_payment_date'] = pd.to_datetime(userbase['last payment date'], errors='coerce')


In [6]:
# Aggregate churn data
churn_summary = userbase.groupby(['subscription type'])['churn'].mean().reset_index()
churn_summary.rename(columns={'churn': 'churn_rate'}, inplace=True)
print(churn_summary.head())

  subscription type  churn_rate
0             Basic         1.0
1           Premium         1.0
2          Standard         1.0


In [7]:
# Viewing Activity Analysis
print("\nViewing Activity Analysis:")
viewing_activity['start_time'] = pd.to_datetime(viewing_activity['start time'], errors='coerce')
viewing_activity['session_duration'] = viewing_activity['duration'].str.split(':').apply(lambda x: int(x[0])*60 + int(x[1]) if len(x) == 2 else 0)
activity_summary = viewing_activity.groupby('profile name')['session_duration'].mean().reset_index()
activity_summary.rename(columns={'session_duration': 'average_session_duration'}, inplace=True)
print(activity_summary.head())



Viewing Activity Analysis:


/var/folders/w6/vkcmr8l91_12vz93c0c46mtm0000gn/T/ipykernel_46806/1084255557.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  viewing_activity['start_time'] = pd.to_datetime(viewing_activity['start time'], errors='coerce')


  profile name  average_session_duration
0       User 0                       0.0
1       User 1                       0.0
2       User 2                       0.0
3       User 3                       0.0
4       User 4                       0.0


In [8]:
# Devices Analysis
print("\nDevices Analysis:")
devices['acct_first_playback_date'] = pd.to_datetime(devices['acct first playback date'], errors='coerce')
devices['acct_last_playback_date'] = pd.to_datetime(devices['acct last playback date'], errors='coerce')
devices['playback_duration_days'] = (devices['acct_last_playback_date'] - devices['acct_first_playback_date']).dt.days
device_summary = devices.groupby('device type')['playback_duration_days'].mean().reset_index()
device_summary.rename(columns={'playback_duration_days': 'average_playback_duration_days'}, inplace=True)
print(device_summary.head())



Devices Analysis:
      device type  average_playback_duration_days
0   Device Type 0                      116.564103
1   Device Type 1                      107.307692
2  Device Type 10                             NaN
3  Device Type 11                      158.000000
4   Device Type 2                       59.555556


In [9]:
# Search History Analysis
print("\nSearch History Analysis:")
if {'profile name', 'country iso code', 'query typed'}.issubset(search_history.columns):
    search_history_analysis = search_history.groupby(['profile name', 'country iso code'])['query typed'].count().reset_index()
    search_history_analysis.rename(columns={'query typed': 'search_count'}, inplace=True)
    print(search_history_analysis.head())
else:
    print("Required columns are missing in search_history.")


Search History Analysis:
  profile name country iso code  search_count
0       User 0               AE            11
1       User 0               IN             7
2       User 1               IN            74
3       User 2               IN            40
4       User 3               IN            17


In [10]:
# Clickstream Analysis
print("\nClickstream Analysis:")
clickstream['click_time'] = pd.to_datetime(clickstream['click utc ts'], errors='coerce')
clickstream['total_clicks'] = clickstream.groupby('profile name')['navigation level'].transform('count')
clickstream_summary = clickstream.groupby('profile name')['total_clicks'].mean().reset_index()
clickstream_summary.rename(columns={'total_clicks': 'average_clicks'}, inplace=True)
print(clickstream_summary.head())


Clickstream Analysis:
  profile name  average_clicks
0       User 0           933.0
1       User 1             6.0
2       User 2             8.0
3       User 3           306.0
4       User 4          1433.0


/var/folders/w6/vkcmr8l91_12vz93c0c46mtm0000gn/T/ipykernel_46806/1661201478.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  clickstream['click_time'] = pd.to_datetime(clickstream['click utc ts'], errors='coerce')
